# Subjective Quiz

## Initialising spark session

In [1]:
from pyspark.sql import SparkSession # Importing SparkSession 
from pyspark.sql.types import IntegerType, StringType

In [2]:
# Initialising Spark session
spark = SparkSession.builder.appName("WholeSaleCustDataParser").getOrCreate()

## Answer to Q1

Reading csv file as data frame using spark.

In [3]:
# The path where file exits
filePath = "Wholesale customers data.csv"

# customerDF = spark.read.csv(path=filePath, header=True, inferSchema=True, 
#                         ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)

# Because of following error, I've to specify the csv format using spark.read.format method
# Error: Multiple sources found for csv (org.apache.spark.sql.execution.datasources.csv.CSVFileFormat,
# com.databricks.spark.csv.DefaultSource15), please specify the fully qualified class name.

customerDF = spark.read.format('com.databricks.spark.csv')\
               .option("header", "true")\
               .load(filePath)


In [4]:
customerDF.show(10)

+-------+------+-----+-----+-------+------+----------------+----------+
|Channel|Region|Fresh| Milk|Grocery|Frozen|Detergents_Paper|Delicassen|
+-------+------+-----+-----+-------+------+----------------+----------+
|      2|     3|12669| 9656|   7561|   214|            2674|      1338|
|      2|     3| 7057| 9810|   9568|  1762|            3293|      1776|
|      2|     3| 6353| 8808|   7684|  2405|            3516|      7844|
|      1|     3|13265| 1196|   4221|  6404|             507|      1788|
|      2|     3|22615| 5410|   7198|  3915|            1777|      5185|
|      2|     3| 9413| 8259|   5126|   666|            1795|      1451|
|      2|     3|12126| 3199|   6975|   480|            3140|       545|
|      2|     3| 7579| 4956|   9426|  1669|            3321|      2566|
|      1|     3| 5963| 3648|   6192|   425|            1716|       750|
|      2|     3| 6006|11093|  18881|  1159|            7425|      2098|
+-------+------+-----+-----+-------+------+----------------+----

In [5]:
# Just checking the spark version
print("Spark Version: {}".format(spark.version))

Spark Version: 2.0.2


## Answer to Q2

See the schema of the DF.

In [6]:
# Printing schema in readable format
customerDF.printSchema()

root
 |-- Channel: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Fresh: string (nullable = true)
 |-- Milk: string (nullable = true)
 |-- Grocery: string (nullable = true)
 |-- Frozen: string (nullable = true)
 |-- Detergents_Paper: string (nullable = true)
 |-- Delicassen: string (nullable = true)



In [7]:
# Alternate method to view schema
customerDF.schema

StructType(List(StructField(Channel,StringType,true),StructField(Region,StringType,true),StructField(Fresh,StringType,true),StructField(Milk,StringType,true),StructField(Grocery,StringType,true),StructField(Frozen,StringType,true),StructField(Detergents_Paper,StringType,true),StructField(Delicassen,StringType,true)))

In [8]:
# Changing the schema of columns

for col in customerDF.columns:
    customerDF = customerDF.withColumn(col, customerDF[col].cast(IntegerType()))

In [9]:
# Printing schema in readable format
customerDF.printSchema()

root
 |-- Channel: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- Fresh: integer (nullable = true)
 |-- Milk: integer (nullable = true)
 |-- Grocery: integer (nullable = true)
 |-- Frozen: integer (nullable = true)
 |-- Detergents_Paper: integer (nullable = true)
 |-- Delicassen: integer (nullable = true)



## Answer to Q3

Use select to view a single column or a set of chosen columns.

In [10]:
# Select only the "Fresh" column
customerDF.select("Fresh").show(10)

+-----+
|Fresh|
+-----+
|12669|
| 7057|
| 6353|
|13265|
|22615|
| 9413|
|12126|
| 7579|
| 5963|
| 6006|
+-----+
only showing top 10 rows



In [11]:
# Selecting multiple columns: Grocery and Frozen
customerDF.select(customerDF.Grocery, customerDF.Frozen).show(10)

+-------+------+
|Grocery|Frozen|
+-------+------+
|   7561|   214|
|   9568|  1762|
|   7684|  2405|
|   4221|  6404|
|   7198|  3915|
|   5126|   666|
|   6975|   480|
|   9426|  1669|
|   6192|   425|
|  18881|  1159|
+-------+------+
only showing top 10 rows



## Answer to Q4

Use filter to see records with fresh sales more than 50000 only.

In [12]:
# fresh sales more than 50000
filteredDF = customerDF.filter(customerDF.Fresh > 50000)

# Checking if I can display all records on console
noOfRows = filteredDF.count()
print("Number of filtered records: {}".format(noOfRows))

filteredDF.show(noOfRows)

Number of filtered records: 7
+-------+------+------+-----+-------+------+----------------+----------+
|Channel|Region| Fresh| Milk|Grocery|Frozen|Detergents_Paper|Delicassen|
+-------+------+------+-----+-------+------+----------------+----------+
|      1|     3| 56159|  555|    902| 10002|             212|      2916|
|      1|     3| 56082| 3504|   8906| 18028|            1480|      2498|
|      1|     3| 76237| 3473|   7102| 16538|             778|       918|
|      1|     3|112151|29627|  18148| 16745|            4948|      8550|
|      1|     1| 56083| 4563|   2124|  6422|             730|      3321|
|      1|     1| 53205| 4959|   7336|  3012|             967|       818|
|      1|     3| 68951| 4411|  12609|  8692|             751|      2406|
+-------+------+------+-----+-------+------+----------------+----------+



## Answer to Q5

Create aggregates on channels and regions variables.

In [13]:
# Aggregates on channels
countChannel = customerDF.groupBy(customerDF.Channel).count()
totalChannel = countChannel.agg({"count": "sum"})

countChannel.show()
totalChannel.show()

+-------+-----+
|Channel|count|
+-------+-----+
|      1|  298|
|      2|  142|
+-------+-----+

+----------+
|sum(count)|
+----------+
|       440|
+----------+



In [14]:
# Function to format and print
def formattedPrint(classType, keys, rows, total):
    print("{}|{}".format(classType.center(15), "Count".center(15)))
    print("-" * 31)
    for row in rows:
        print("{}|{}".format(keys[row[0]].center(15),
                             str(row["count"]).center(15)))
    print("-" * 31)
    print("{}|{}".format("Total".center(15), 
                     str(total.first()["sum(count)"]).center(15)))

In [15]:
# Printing in readable format
# customers Channel - Horeca (Hotel/Restaurant/Cafe) as 1 and 
#                   - Retail channel as 2
channeldict = {1:"Horeca", 2:"Retail"}

formattedPrint("Channel", channeldict, countChannel.collect(), totalChannel)

    Channel    |     Count     
-------------------------------
     Horeca    |      298      
     Retail    |      142      
-------------------------------
     Total     |      440      


In [16]:
# Aggregates on Regions
countRegion = customerDF.groupBy(customerDF.Region).count().orderBy("Region")
totalRegion = countChannel.agg({"count": "sum"})

countRegion.show()
totalRegion.show()

+------+-----+
|Region|count|
+------+-----+
|     1|   77|
|     2|   47|
|     3|  316|
+------+-----+

+----------+
|sum(count)|
+----------+
|       440|
+----------+



In [17]:
# Printing in readable format
# REGION: Lisbon as 1
#         Oporto as 2
#         Other  as 3
regiondict = {1:"Lisbon", 2:"Oporto", 3:"Other"}

formattedPrint("Region", regiondict, countRegion.collect(), totalRegion)

     Region    |     Count     
-------------------------------
     Lisbon    |       77      
     Oporto    |       47      
     Other     |      316      
-------------------------------
     Total     |      440      


## Answer to Q6

Use describe to see summary statistics on dataframe.

In [18]:
import pandas
colsToDescribe = ['Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicassen']
describeData = customerDF.describe(colsToDescribe)

In [19]:
# Changing to pandas dataframe so that I can transpose and display output of describe
describeData.toPandas().set_index('summary').transpose()

summary,count,mean,stddev,min,max
Fresh,440,12000.297727272728,12647.328865076885,3,112151
Milk,440,5796.265909090909,7380.3771745708445,55,73498
Grocery,440,7951.277272727273,9503.162828994346,3,92780
Frozen,440,3071.931818181818,4854.673332592367,25,60869
Detergents_Paper,440,2881.4931818181817,4767.854447904201,3,40827
Delicassen,440,1524.8704545454545,2820.1059373693965,3,47943


> #### Insights
> - Sale of Fresh products is highest

## Answer to Q7

Change datatype of Channels to Strings.

In [20]:
# Changing datatype of channels
customerDF.withColumn('Channel', 
                      customerDF.Channel.cast(StringType())).printSchema()

root
 |-- Channel: string (nullable = true)
 |-- Region: integer (nullable = true)
 |-- Fresh: integer (nullable = true)
 |-- Milk: integer (nullable = true)
 |-- Grocery: integer (nullable = true)
 |-- Frozen: integer (nullable = true)
 |-- Detergents_Paper: integer (nullable = true)
 |-- Delicassen: integer (nullable = true)



## Answer to Q8

Perform rollups on channels and regions.

In [21]:
# Performing rollup using dataframe rollup function.
customerDF.rollup("Channel", "Region").count().orderBy("Channel", "Region").show()

# Channel: Horeca (Hotel/Restaurant/Cafe) as 1 and 
#          Retail as 2
# REGION: Lisbon as 1
#         Oporto as 2
#         Other  as 3

+-------+------+-----+
|Channel|Region|count|
+-------+------+-----+
|   null|  null|  440|
|      1|  null|  298|
|      1|     1|   59|
|      1|     2|   28|
|      1|     3|  211|
|      2|  null|  142|
|      2|     1|   18|
|      2|     2|   19|
|      2|     3|  105|
+-------+------+-----+



> #### Insights
> - The total clients of a wholesale distributor are 440 (Assuming each row is different client)
> - 59 of these clients that are in Lisbon are either Hotel/Restaurant/Cafe and only 18 are Retail
> - 28 of these clients that are in Oporto are either Hotel/Restaurant/Cafe and only 19 are Retail
> - 211 of the remaining clients from Other regions are Hotel/Restaurant/Cafe and 105 are Retail
----
> Reference: Channel - Horeca (Hotel/Restaurant/Cafe) as 1 and Retail channel as 2; 
> REGION: Lisbon as 1, Oporto as 2 and Other  as 3